In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DATA_PATH = "/content/drive/MyDrive/datathon/data/"

SEED = 42

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
df = pd.read_csv(f"{DATA_PATH}base_data_forreal.csv")

<ipython-input-125-2ba9eec16305>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{DATA_PATH}base_data_forreal.csv")


In [ ]:
train_tr = pd.read_csv(f"{DATA_PATH}teams_train(analyze).csv") # 학습용 승부예측 데이터
#train_target = pd.read_csv(f"{DATA_PATH}train_target_1226.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}teams_test(analyze).csv") # 테스트용 승부예측 데이터
#test_target = pd.read_csv(f"{DATA_PATH}test_target_1226.csv") # 테스트용 정답 데이터

In [ ]:
train_tr.shape, test_tr.shape

((10000, 156), (2334, 111))

# 전처리

In [ ]:
drop_list_team = [
    "participantid",
    "playername",
    'position',
    "champion",
    "teamkills",
    "teamdeaths",
    "firstbloodkill",
    "firstbloodassist",
    "firstbloodvictim",
    "dragons (type unknown)",
    "damageshare",
    "earnedgoldshare",
    "total cs",
    "monsterkillsownjungle",
    "monsterkillsenemyjungle",
    "goldat20",
    "xpat20",
    "csat20",
    "opp_goldat20",
    "opp_xpat20",
    "opp_csat20",
    "golddiffat20",
    "xpdiffat20",
    "csdiffat20",
    "killsat20",
    "assistsat20",
    "deathsat20",
    "opp_killsat20",
    "opp_assistsat20",
    "opp_deathsat20",
    "goldat25",
    "xpat25",
    "csat25",
    "opp_goldat25",
    "opp_xpat25",
    "opp_csat25",
    "golddiffat25",
    "xpdiffat25",
    "csdiffat25",
    "killsat25",
    "assistsat25",
    "deathsat25",
    "opp_killsat25",
    "opp_assistsat25",
    "opp_deathsat25"
]

In [ ]:
train_ft = train_tr.drop(columns=drop_list_team)
test_ft = test_tr.copy()

- teams

In [ ]:
fill_zero_cols = [
    "void_grubs",
    "opp_void_grubs",
]

fill_unknown_cols = [
    "split",
    "ban1",
    "ban2",
    "ban3",
    "ban4",
    "ban5",
]

train_ft[fill_zero_cols] = train_ft[fill_zero_cols].fillna(0)
test_ft[fill_zero_cols] = test_ft[fill_zero_cols].fillna(0)

train_ft[fill_unknown_cols] = train_ft[fill_unknown_cols].fillna("unknown")
test_ft[fill_unknown_cols] = test_ft[fill_unknown_cols].fillna("unknown")

train_ft.dropna(subset=['pick1'], inplace=True)
test_ft.dropna(subset=['pick1'], inplace=True)

- lpl 결측치

In [ ]:
lpl_team = [
    "Anyone's Legend",
    "Bilibili Gaming",
    "EDward Gaming",
    "FunPlus Phoenix",
    "Invictus Gaming",
    "JD Gaming",
    "LGD Gaming",
    "LNG Esports",
    "Oh My God",
    "Rare Atom",
    "Royal Never Give Up",
    "Team WE",
    "Top Esports",
    "ThunderTalk Gaming",
    "Ultra Prime",
    "Weibo Gaming",
    "Ninjas in Pyjamas"
]

In [ ]:
missing_columns = train_ft.columns[train_ft.isnull().sum() > 0]
null_samples = train_ft[train_ft[missing_columns].isnull().any(axis=1)]

for idx, row in null_samples.iterrows():
    team_history = train_ft[
        (train_ft["teamname"] == row["teamname"])
        & (train_ft["gameid"] < row["gameid"])
    ].sort_values("gameid", ascending=False)

    for col in missing_columns:
        if pd.isnull(row[col]):
            if row["teamname"] in lpl_team:
                # LPL 팀인 경우 LCK 평균으로 채움
                lck_mean = train_ft[train_ft["league"] == "LCK"][col].mean()
                train_ft.loc[idx, col] = lck_mean
            else:
                # LPL 팀이 아닌 경우 해당 팀의 이전 5경기 평균으로 채움
                prev_5_mean = team_history[col].head(5).mean()
                if pd.notnull(prev_5_mean):
                    train_ft.loc[idx, col] = prev_5_mean
                else:
                    # 이전 5경기 데이터가 없는 경우 해당 팀의 전체 평균으로 채움
                    team_mean = train_ft[train_ft["teamname"] == row["teamname"]][col].mean()
                    train_ft.loc[idx, col] = team_mean

In [ ]:
train_ft.isnull().sum().sum()

0

In [ ]:
missing_columns = test_ft.columns[test_ft.isnull().sum() > 0]
null_samples = test_ft[test_ft[missing_columns].isnull().any(axis=1)]

for idx, row in null_samples.iterrows():
    team_history = train_ft[
        (train_ft["teamname"] == row["teamname"])
        & (train_ft["gameid"] < row["gameid"])
    ].sort_values("gameid", ascending=False)

    for col in missing_columns:
        if pd.isnull(row[col]):
            if row["teamname"] in lpl_team:
                # LPL 팀인 경우 학습 데이터의 LCK 평균으로 채움
                lck_mean = train_ft[train_ft["league"] == "LCK"][col].mean()
                test_ft.loc[idx, col] = lck_mean
            else:
                # LPL 팀이 아닌 경우 학습 데이터에서 해당 팀의 이전 5경기 평균으로 채움
                prev_5_mean = team_history[col].head(5).mean()
                if pd.notnull(prev_5_mean):
                    test_ft.loc[idx, col] = prev_5_mean
                else:
                    # 이전 5경기 데이터가 없는 경우 학습 데이터에서 해당 팀의 전체 평균으로 채움
                    team_mean = train_ft[train_ft["teamname"] == row["teamname"]][col].mean()
                    test_ft.loc[idx, col] = team_mean

In [ ]:
test_ft.isnull().sum().sum()

0

In [ ]:
missing_columns = test_ft.columns[test_ft.isnull().sum() > 0]
missing_columns

Index([], dtype='object')

In [ ]:
train_ft.shape, test_ft.shape

((9913, 111), (2324, 111))

# feature

- 연도별 팀순위

In [ ]:
train_ft["date"] = pd.to_datetime(train_ft["date"])

train_ft["year"] = train_ft["date"].dt.year

In [ ]:
def calculate_team_rankings(df):
    """
    팀의 결과에 따라 연도별 리그별 팀 순위를 계산합니다.

    Args:
        df: 팀 결과 데이터가 포함된 DataFrame. 'year', 'league', 'teamname', 'result' 열이 필요합니다.
            'result' 열은 팀의 승패를 나타내는 값 (예: 승리=1, 패배=0)입니다.

    Returns:
        연도별, 리그별 팀 순위가 포함된 DataFrame.
    """

    rankings = []
    for year in df['year'].unique():
        for league in df['league'].unique():
            year_league_df = df[(df['year'] == year) & (df['league'] == league)]
            if year_league_df.empty:
                continue

            # 승리 횟수를 기준으로 정렬
            team_wins = year_league_df.groupby('teamname')['result'].sum().reset_index()
            team_wins = team_wins.sort_values(by='result', ascending=False)

            # 순위 부여
            team_wins['rank'] = range(1, len(team_wins) + 1)

            # 연도, 리그 정보 추가
            team_wins['year'] = year
            team_wins['league'] = league

            rankings.append(team_wins)

    return pd.concat(rankings)


# 데이터프레임 예시 (실제 데이터로 대체해야 함)
# 'year', 'league', 'teamname', 'result' 열이 존재해야 합니다.
# 'result' 열은 팀의 승리/패배를 나타내는 값 (예: 승리=1, 패배=0)입니다.


# 함수 호출 및 결과 출력
team_rankings = calculate_team_rankings(train_ft)
team_rankings

,teamname,result,rank,year,league
9,T1,76,1,2022,LCK
3,Gen.G,73,2,2022,LCK
2,Dplus KIA,61,3,2022,LCK
1,DRX,47,4,2022,LCK
5,KT Rolster,46,5,2022,LCK
...,...,...,...,...,...
7,Team Whales,19,4,2024,VCS
0,CERBERUS Esports,12,5,2024,VCS
2,MGN Blue Esports,12,6,2024,VCS
5,Team Flash,12,7,2024,VCS


In [171]:
# 연도별 팀 평균kda 계산(팀일경우 deaths가 0인값이 없어서 별도 처리 안함)
def calculate_yearly_team_kda(df):
    # KDA 계산
    df['KDA'] = (df['kills'] + df['assists']) / df['deaths']
    df['KDA'] = df['KDA'].replace(np.inf, 0)  # 무한대 값 처리

    # 연도별 팀 평균 KDA 계산
    yearly_team_kda = df.groupby(['year', 'teamname'])['KDA'].mean().reset_index()
    return yearly_team_kda

yearly_team_kda = calculate_yearly_team_kda(train_ft)
yearly_team_kda

,year,teamname,KDA
0,2022,100 Thieves,7.105605
1,2022,AS Esports,2.209044
2,2022,Astralis,3.926283
3,2022,BNK FEARX,4.897943
4,2022,Beyond Gaming,7.031276
...,...,...,...
195,2024,Team Whales,4.112147
196,2024,V3 Esports,1.390224
197,2024,Vivo Keyd Stars,6.385573
198,2024,West Point Esports,2.028894
